<a href="https://colab.research.google.com/github/sebbyjp/mbodied-agents/blob/prod/diffuser_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Setup

apt install requirements

In [ ]:
!apt-get install -y \
    libgl1-mesa-dev \
    libgl1-mesa-glx \
    libglew-dev \
    libosmesa6-dev \
    software-properties-common

!apt-get install -y patchelf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libglew-dev is already the newest version (2.0.0-5).
libgl1-mesa-dev is already the newest version (20.0.8-0ubuntu1~18.04.1).
libgl1-mesa-glx is already the newest version (20.0.8-0ubuntu1~18.04.1).
libosmesa6-dev is already the newest version (20.0.8-0ubuntu1~18.04.1).
software-properties-common is already the newest version (0.96.24.32.18).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 4 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
patchelf is already the newest version (0.9-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 4 not upgraded.


Clone repo and install dependencies

In [ ]:
%cd /content

!git clone https://github.com/jannerm/diffuser.git
%cd diffuser
!git checkout 06b8e6a042e6a3312d50ed8048cba14afeab3085
%cd /content
!curl -L -o diffuser-pretrained.tar https://www.dropbox.com/s/1x73p8x66mj7dvk/diffuser-pretrained.tar?dl=1
!tar -xf diffuser-pretrained.tar
!mv diffuser-pretrained/logs diffuser/logs
!rm -r diffuser-pretrained*

%cd diffuser
%pip install -f https://download.pytorch.org/whl/torch_stable.html \
                typed-argument-parser \
                scikit-image==0.17.2 \
                scikit-video==1.1.11 \
                gitpython \
                einops \
                pillow \
                free-mujoco-py \
                gym==0.18.0 \
                git+https://github.com/JannerM/d4rl.git@0e84a4d29db3ae501043215ce1d91843929f1949 \
                git+https://github.com/aravindr93/mjrl

%pip install -e .

/content
fatal: destination path 'diffuser' already exists and is not an empty directory.
/content/diffuser
HEAD is now at 06b8e6a Update README.md
/content
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   130    0   130    0     0    343      0 --:--:-- --:--:-- --:--:--   343
100   363  100   363    0     0    449      0 --:--:-- --:--:-- --:--:--     0
100  747M  100  747M    0     0  18.4M      0  0:00:40  0:00:40 --:--:-- 28.6M
mv: cannot move 'diffuser-pretrained/logs' to 'diffuser/logs/logs': Directory not empty
/content/diffuser
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Cloning https://github.com/JannerM/d4rl.git (to revision 0e84a4d29db3ae501043215ce1d91843929f1949) to /tmp/pip-req-build-nd0b2neb
  Running command git clone -q https://git

In [ ]:
## cythonize mujoco-py at first import
import diffuser.utils as utils

# import importlib
# importlib.reload(utils.colab)

### Conditional sampling

In [ ]:
class Args:
  loadpath = 'logs/pretrained/hopper-medium-expert-v2/diffusion/H128_T100'
  diffusion_epoch = 'latest'
  n_samples = 4
  device = 'cuda:0'

args = Args()

Load diffusion model from disk

In [ ]:
%cd /content/diffuser

diffusion_experiment = utils.load_diffusion(
    args.loadpath, epoch=args.diffusion_epoch)

dataset = diffusion_experiment.dataset
renderer = diffusion_experiment.renderer
model = diffusion_experiment.trainer.ema_model

/content/diffuser
[ utils/serialization ] Loaded config from logs/pretrained/hopper-medium-expert-v2/diffusion/H128_T100/dataset_config.pkl

[utils/config ] Config: <class 'diffuser.datasets.sequence.SequenceDataset'>
    env: hopper-medium-expert-v2
    horizon: 128
    max_path_length: 1000
    normalizer: LimitsNormalizer
    preprocess_fns: []
    use_padding: True

[ utils/serialization ] Loaded config from logs/pretrained/hopper-medium-expert-v2/diffusion/H128_T100/render_config.pkl

[utils/config ] Config: <class 'diffuser.utils.rendering.MuJoCoRenderer'>
    env: hopper-medium-expert-v2

[ utils/serialization ] Loaded config from logs/pretrained/hopper-medium-expert-v2/diffusion/H128_T100/model_config.pkl

[utils/config ] Config: <class 'diffuser.models.temporal.TemporalUnet'>
    cond_dim: 11
    dim_mults: (1, 4, 8)
    horizon: 128
    transition_dim: 14

[ utils/serialization ] Loaded config from logs/pretrained/hopper-medium-expert-v2/diffusion/H128_T100/diffusion_config.p

RuntimeError: ignored

Initialize environment

In [ ]:
env = dataset.env
obs = env.reset()

Run reverse diffusion

In [ ]:
diffusion_chain = utils.colab.run_diffusion(
    model, dataset, obs, args.n_samples, args.device
)
print(diffusion_chain.shape)

Render the samples

In [ ]:
samples = diffusion_chain[-1]
utils.colab.show_sample(renderer, samples)

Render the full reverse diffusion process for the first sample.

This renders `horizon * n_diffusion_steps` images, so takes a few minutes. Increasing `substep` will speed up the video generation, but cause it to skip frames.

In [ ]:
utils.colab.show_diffusion(renderer, diffusion_chain[:,:1], substep=1)

Now let's run diffusion with different horizons

In [ ]:
horizons = [16, 128, 256, 512, 1024]
diffusion_chains = [
      utils.colab.run_diffusion(
          model, dataset, obs, n_samples=1, device=args.device, horizon=horizon
      )
      for horizon in horizons
]

for horizon, diffusion_chain in zip(horizons, diffusion_chains):
  print(f'horizon = {horizon} | samples shape = {diffusion_chain.shape}')

The default model was trained on trajectories of length 128, but can produce trajectories of any length in a constant number of diffusion steps

In [ ]:
utils.colab.show_samples(renderer, diffusion_chains)